In [80]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline
import scipy.stats
from sklearn.metrics.pairwise import cosine_similarity

SyntaxError: invalid syntax (<ipython-input-80-5921ca276a8d>, line 8)

In [2]:
train = pd.read_csv('train.csv')
print(train.head())
print(f'The shape of the train data is:{train.shape}')
print(f'Number of null values:{train.isna().sum()}')
print(f'Number of unique values:{train.nunique()}')

   userId  movieId  rating   timestamp
0    5163    57669     4.0  1518349992
1  106343        5     4.5  1206238739
2  146790     5459     5.0  1076215539
3  106362    32296     2.0  1423042565
4    9041      366     3.0   833375837
The shape of the train data is:(10000038, 4)
Number of null values:userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
Number of unique values:userId        162541
movieId        48213
rating            10
timestamp    8795101
dtype: int64


In [3]:
test = pd.read_csv('test.csv')
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [4]:
genome_score = pd.read_csv('genome_scores.csv').dropna()
genome_score.head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [5]:
genome_tag = pd.read_csv('genome_tags.csv').dropna()
genome_tag.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [6]:
links = pd.read_csv('links.csv').dropna()
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
tag = pd.read_csv('tags.csv').dropna()
tag.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [8]:
movies = pd.read_csv('movies.csv').dropna()
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [9]:
movie_year = []
for movie in movies['title']:
    yr = movie[-6:]
    movie_year.append(yr)
    
movies['movie_year'] = pd.DataFrame(movie_year)
movies.head()

,movieId,title,genres,movie_year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,(1995)
4,5,Father of the Bride Part II (1995),Comedy,(1995)


In [10]:
imdb = pd.read_csv('imdb_data.csv').dropna()
imdb.head()

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


#### Merging

In [14]:
genomes = genome_score.merge(genome_tag, on='tagId')
genomes.head()

,movieId,tagId,relevance,tag
0,1,1,0.02875,007
1,2,1,0.04125,007
2,3,1,0.04675,007
3,4,1,0.03425,007
4,5,1,0.04300,007


In [ ]:
movies_train = movies.merge(train, on='movieId', how = 'inner')
movies_train.head()

In [26]:
movies_train.shape

(10000038, 7)

In [30]:
#grouping by movie title
rating_agg = movies_train.groupby('title').agg(mean_rating = ('rating', 'mean'), 
                                               number_of_rating= ('rating', 'count')).reset_index()

In [92]:
rating_agg_3000 = rating_agg[rating_agg['number_of_rating'] > 10000]
rating_agg_3000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 287 to 47469
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             127 non-null    object 
 1   mean_rating       127 non-null    float64
 2   number_of_rating  127 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.0+ KB


In [93]:
rating_agg_3000.sort_values(by = 'number_of_rating', ascending = False).head()

,title,mean_rating,number_of_rating
34480,"Shawshank Redemption, The (1994)",4.417624,32831
14537,Forrest Gump (1994),4.047216,32383
31236,Pulp Fiction (1994),4.195097,31697
34840,"Silence of the Lambs, The (1991)",4.144172,29444
25144,"Matrix, The (1999)",4.154253,29014


In [94]:
df = pd.merge(movies_train, rating_agg_3000[['title']], on = 'title', how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1959734 entries, 0 to 1959733
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   movieId     int64  
 1   title       object 
 2   genres      object 
 3   movie_year  object 
 4   userId      int64  
 5   rating      float64
 6   timestamp   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 119.6+ MB


#### creating user matrix

In [105]:
matrix = df.pivot_table(index = 'userId', columns = 'title', values = 'rating').reset_index(drop=True)
matrix.tail()

title,2001: A Space Odyssey (1968),Ace Ventura: Pet Detective (1994),Aladdin (1992),Alien (1979),Aliens (1986),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),Apocalypse Now (1979),Apollo 13 (1995),...,"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),Up (2009),"Usual Suspects, The (1995)",V for Vendetta (2006),WALL·E (2008),Waterworld (1995),Willy Wonka & the Chocolate Factory (1971),X-Men (2000)
155908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155910,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
155912,3.0,0.5,NaN,4.0,NaN,4.5,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [96]:
matrix.shape

(155913, 127)

In [100]:
matrix_norm = matrix.subtract(matrix.mean(axis = 1), axis = 'rows')
matrix_norm.head()

title,2001: A Space Odyssey (1968),Ace Ventura: Pet Detective (1994),Aladdin (1992),Alien (1979),Aliens (1986),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),Apocalypse Now (1979),Apollo 13 (1995),...,"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),Up (2009),"Usual Suspects, The (1995)",V for Vendetta (2006),WALL·E (2008),Waterworld (1995),Willy Wonka & the Chocolate Factory (1971),X-Men (2000)
0,-0.074803,-0.074803,-0.074803,-0.074803,-0.074803,4.425197,-0.074803,-0.074803,-0.074803,-0.074803,...,-0.074803,-0.074803,-0.074803,-0.074803,-0.074803,-0.074803,-0.074803,-0.074803,-0.074803,-0.074803
1,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,...,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,4.503937
2,4.011811,-0.988189,-0.988189,-0.988189,3.011811,-0.988189,-0.988189,4.011811,3.011811,-0.988189,...,3.011811,3.511811,-0.988189,-0.988189,4.011811,3.011811,-0.988189,-0.988189,-0.988189,-0.988189
3,3.511811,-0.488189,-0.488189,2.011811,3.011811,-0.488189,-0.488189,-0.488189,-0.488189,-0.488189,...,-0.488189,-0.488189,-0.488189,4.011811,-0.488189,-0.488189,-0.488189,-0.488189,-0.488189,1.011811
4,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063,4.503937,...,-0.496063,4.503937,2.503937,-0.496063,4.503937,-0.496063,-0.496063,-0.496063,-0.496063,-0.496063


In [107]:
user_similarity = matrix_norm.T.corr()
user_similarity.head()
# user_similarity = cosine_similarity(matrix_norm.fillna(0))
# user_matrix = pd.DataFrame(user_similarity)
# user_matrix.head

MemoryError: Unable to allocate 181. GiB for an array with shape (155913, 155913) and data type float64